In [1]:
!pip install dash jupyter-dash plotly pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 9.8 MB/s eta 0:00:00a 0:00:01


In [8]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

In [3]:
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv")
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month_name()

In [9]:
app = dash.Dash(__name__)
app.title = "XYZAutomotives Dashboard"

app.layout = html.Div([
    html.H1("XYZAutomotives Sales Dashboard", style={'textAlign': 'center'}),

    html.Div([
        html.Div([
            html.Label("Select Report Type"),
            dcc.Dropdown(
                id='report-dropdown',
                options=[
                    {'label': 'Yearly Statistics', 'value': 'yearly'},
                    {'label': 'Recession Period Statistics', 'value': 'recession'}
                ],
                value='yearly'
            )
        ], style={'width': '45%', 'display': 'inline-block', 'padding': '10px'}),

        html.Div([
            html.Label("Select Year"),
            dcc.Dropdown(
                id='year-dropdown',
                options=[{'label': y, 'value': y} for y in sorted(df['Year'].unique())],
                value=df['Year'].min()
            )
        ], style={'width': '45%', 'display': 'inline-block', 'padding': '10px'}),
    ]),

    html.Div([
        html.Div([dcc.Graph(id='plot1')], style={'width': '48%', 'display': 'inline-block'}),
        html.Div([dcc.Graph(id='plot2')], style={'width': '48%', 'display': 'inline-block'}),
    ]),
    html.Div([
        html.Div([dcc.Graph(id='plot3')], style={'width': '48%', 'display': 'inline-block'}),
        html.Div([dcc.Graph(id='plot4')], style={'width': '48%', 'display': 'inline-block'}),
    ])
])


In [10]:
@app.callback(
    Output('year-dropdown', 'disabled'),
    Input('report-dropdown', 'value')
)
def toggle_year_dropdown(report_type):
    return report_type != 'yearly'


In [11]:
@app.callback(
    [Output('plot1', 'figure'),
     Output('plot2', 'figure'),
     Output('plot3', 'figure'),
     Output('plot4', 'figure')],
    [Input('report-dropdown', 'value'),
     Input('year-dropdown', 'value')]
)
def update_graphs(report_type, selected_year):
    if report_type == 'yearly':
        df_year = df[df['Year'] == selected_year]

        fig1 = px.line(df.groupby("Year")["Automobile_Sales"].mean().reset_index(),
                       x="Year", y="Automobile_Sales", title="Avg Sales by Year")

        fig2 = px.line(df_year.groupby("Month")["Automobile_Sales"].sum().reset_index(),
                       x="Month", y="Automobile_Sales", title=f"Monthly Sales in {selected_year}")

        fig3 = px.bar(df_year.groupby("Vehicle_Type")["Automobile_Sales"].mean().reset_index(),
                      x="Vehicle_Type", y="Automobile_Sales", title=f"Avg Sales per Vehicle Type in {selected_year}")

        fig4 = px.pie(df_year.groupby("Vehicle_Type")["Advertising_Expenditure"].sum().reset_index(),
                      names="Vehicle_Type", values="Advertising_Expenditure",
                      title=f"Ad Spend by Vehicle Type in {selected_year}")
    else:
        df_rec = df[df["Recession"] == 1]

        fig1 = px.line(df_rec.groupby("Year")["Automobile_Sales"].mean().reset_index(),
                       x="Year", y="Automobile_Sales", title="Sales During Recession (Year-wise)")

        fig2 = px.bar(df_rec.groupby("Vehicle_Type")["Automobile_Sales"].mean().reset_index(),
                      x="Vehicle_Type", y="Automobile_Sales", title="Avg Sales by Vehicle Type During Recession")

        fig3 = px.pie(df_rec.groupby("Vehicle_Type")["Advertising_Expenditure"].sum().reset_index(),
                      names="Vehicle_Type", values="Advertising_Expenditure",
                      title="Ad Expenditure Share During Recession")

        fig4 = px.bar(df_rec.groupby("Vehicle_Type")["unemployment_rate"].mean().reset_index(),
                      x="Vehicle_Type", y="unemployment_rate", title="Unemployment Rate by Vehicle Type")

    return fig1, fig2, fig3, fig4


In [13]:
app.run(debug=True)